# Building a spam filter for SMS messages

In this project, the goal is to teach the computer how to classify new messages. I am going to use the multinomial Naive Bayes algorithm and a dataset of 5,572 SMS messages already classified by humans. The spam filter should have an accuracy greater than 80%.

The dataset was put together by Tiago A. Almeida and Jose Maria Gomez Hidalgo and can be downloaded [here](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) or [here](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection).
For more information about the data collection process check [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition).



In [1]:
import pandas as pd
sms_spam = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

In [2]:
sms_spam.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms_spam.shape

(5572, 2)

In [4]:
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

Over 86% of the messages are non-spam and 13% are spams.

# Training set and test set

The spam filter will be successful if I manage to have an accuracy of over 80%.
I will make sure of that by testing it at the end, so let's start by splitting the dataset into 2:
- a training set (about 80% of the data set -  4,458 messages) to use for training our computer on how to classify new messages
- a test set (about 20% - 1,114 messages) which I will use to test how good the spam filter is with classifying new messages.

All 1,114 messages have been cassified by humans, but when I am going to use this set as test and treat them as new and have the filter classify them. This way I will be able to compare the algorithm classification and the human classification to test the accuracy of the filter.

I will start by randomizing the entire dataset, then split it into training set and test set.

In [5]:
randomized = sms_spam.sample(frac=1, random_state=1)
training_test_index = round(len(randomized)*0.8)
training_set = randomized[:training_test_index].reset_index(drop=True)
test_set = randomized[training_test_index:].reset_index(drop=True)

In [6]:
training_set.shape


(4458, 2)

In [7]:
test_set.shape

(1114, 2)

Let's check the percentages of spam vs ham in both data sets. I need them to be similar to what I have in the full data set.

In [8]:
training_set['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [9]:
test_set['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

# Data cleaning

In order to use the Naive Bayes algorithm (that makes the classification based on some equations that use the number of times one word occures in spam/non-spam messages), I will have to do some data cleaning, and bring the data to a number format.

Ex. 

||Label|SMS|
|---|-----|----|
|0|spam |SECRET Prize! Claim secret prize now!!|
|1|ham|Coming to my secret party?|
|2|spam|Winner! Claim secret prize now!

should be:

||Label|secret|prize|claim|now|coming|to|my|party|winner|
|-|-|-|-|-|-|-|-|-|-|-|
|0|spam|2|2|1|1|0|0|0|0|0|
|1|ham|1|0|0|0|1|1|1|1|0|
|2|spam|1|1|1|1|0|0|0|0|1|

Here are the transformations I need to make:
- get rid of the sms column and replace it with new columns that represent a unique word from the vocabulary
- each row describes a single message, with values that tell us if the message is a spam or not and how many times each word occurs 
- all words will be lower case and will be counted as such (SECRET is the same with secret)
- the punctuation is not taken into account

# Letter case and punctuation

In [10]:
training_set['SMS'] = training_set['SMS'].str.replace('\W',' ')
training_set['SMS'] = training_set['SMS'].str.lower()

# Creating the vocabulary
A Python list of unique words from our training set.

In [11]:
training_set['SMS'] = training_set['SMS'].str.split()
vocabulary = []

for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)

vocabulary = list(set(vocabulary))

len(vocabulary)

7783

# Training Set transformation
Build a dictionary and then convert it into a DataFrame.
Concatenate the newly created DataFrame with the old one, to include the label and sms columns.

In [12]:
word_count_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_count_per_sms[word][index] += 1

In [13]:
word_count = pd.DataFrame(word_count_per_sms)

In [14]:
training_set_final = pd.concat([training_set, word_count], axis=1)
training_set_final.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


# Create the spam filter

In order to be able to classify new messages, the Naive Bayes algorithm will need to know the probability values of the two equations below:
- P(Spam|$w_1$,$w_2$,...,$w_n$) $∝$ P(Spam) * $∏_{i=1}^n$ P($w_i$|Spam)
- P(Ham|$w_1$,$w_2$,...,$w_n$) $∝$ P(Ham) * $∏_{i=1}^n$ P($w_i$|Ham)

To calculate P($w_i$|Spam) and P($w_i$|Ham) I need to use these equations:
 $$P(w_i|Spam) = \frac {N_{w_i|Spam} + ∝}{N_{Spam} + ∝ *  N_{Vocabulary}}$$


 $$P(w_i|Ham) = \frac {N_{w_i|Ham} + ∝}{N_{Ham} + ∝ * N_{Vocabulary}}$$
 
 $N_{Spam}$ is equal to the number of words in all(not just unique words) the spam messages.
 
 $N_{Ham}$ is equal to the number of words in all(not just unique words) the non-spam messages.
 
 I will use the Laplace smoothing and set $∝$ =1.
 
 Let's start by calculating the constants.

In [15]:
spam = training_set_final[training_set_final['Label']=='spam']
ham = training_set_final[training_set_final['Label']=='ham']

p_spam = len(spam)/len(training_set_final)
p_ham = len(ham)/len(training_set_final)

n_words_per_spam = spam['SMS'].apply(len)
n_spam = n_words_per_spam.sum()
n_words_per_ham = ham['SMS'].apply(len)
n_ham = n_words_per_ham.sum()

n_vocabulary = len(vocabulary)

alpha = 1


I will proceed with calculating the parameters P($w_i$|Spam) and P($w_i$|Ham).
Each parameter is a conditional probability value associated with each word in the dictionary.
I have a total of 7,783 words in our vocabulary and I will need to calculate a total of 15,566 probabilities for both parameters.

In [16]:
# initialize parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# calculate parameters
for word in vocabulary:
    n_word_given_spam =  spam[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha * n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha * n_vocabulary)
    parameters_ham[word] = p_word_given_ham

# Classifying a new message 

I will now create the spam filter. This will be a function that:
- Takes the input a new message
- Calculates P(Spam|$w_1$, $w_2$, ..., $w_n$) and P(Ham|$w_1$, $w_2$, ..., $w_n$)
- Compares the values of P(Spam|$w_1$, $w_2$, ..., $w_n$) and P(Ham|$w_1$, $w_2$, ..., $w_n$):
     - If $P(Ham|w_1, w_2, ..., w_n) > P(Spam|w_1, w_2, ..., w_n)$ the message is classified as ham;
     - If $P(Ham|w_1, w_2, ..., w_n) < P(Spam|w_1, w_2, ..., w_n)$ the message is classified as spam;
     - If $P(Ham|w_1, w_2, ..., w_n) =  P(Spam|w_1, w_2, ..., w_n)$ the algorithm may request human help. 

In [17]:
import re
def classify(message):
    message = re.sub('\W', ' ', message)
    message= message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)
    
    if p_ham_given_message > p_spam_given_message: 
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message: 
        print('Label: Spam')
    else: 
        print('Equal probabilities, have a human classify this!')
    

Test two messages:
- Sounds good, Tom, then see u there
- WINNER!! This is the secret code to unlock the money: C3421

In [18]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


In [19]:

classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


# Measuring the Spam filter's accuracy

I will now use our test set. 
I will use the classify function to return labels (Spam or Ham) instead of printing them.

In [22]:
def classify_test_set(message):
    message = re.sub('\W', ' ', message)
    message= message.lower()
    message = message.split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
        
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'
    

I will use the function to create a new column in our test set.

In [23]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


# Testing accuracy function

In [24]:
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct +=1
        
print('Correct', correct)
print('Incorrect', total - correct)
print('Accuracy', correct/total)
    

Correct 1100
Incorrect 14
Accuracy 0.9874326750448833


The accuracy is over 98%, that's very good. It means our function managed to classify correctly 1,100 messages out of 1,114.

# Conclusion
In this project I managed to build a spam filter for SMS messages using the Naive Bayes algorithm. The goal to have at least 80% accuracy was overachieved, I managed to obtain over 98% accuracy.

One of the next steps could be checking the messages that were classified inaccurately and try to improve our function. 